# Scrape

In [2]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
from webdriver_manager.chrome import ChromeDriverManager
import pymongo

In [4]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [C:\Users\Paris\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


### Mars News

In [6]:
# Open browser to NASA Mars News Site
browser.visit('https://mars.nasa.gov/news/')

In [7]:
html = browser.html
soup = bs(html, 'html.parser')

# Search for news titles
title_results = soup.find_all('div', class_='content_title')

# Search for paragraph text under news titles
p_results = soup.find_all('div', class_='article_teaser_body')

# Extract first title and paragraph, and assign to variables
news_title = title_results[0].text
news_p = p_results[0].text

print(news_title)
print(news_p)

Mars Now
The Red Planet rotorcraft will shift focus from proving flight is possible on Mars to demonstrating flight operations that future aerial craft could utilize.


### Mars Featured Imaged

In [8]:
# Open browser to JPL Featured Image
browser.visit('https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html')

In [9]:
# Click through to find full image
featured_img = browser.links.find_by_partial_text('FULL IMAGE')

In [10]:
html = browser.html
soup = bs(html, 'html.parser')

# Search for image source
results = soup.find_all("div", class_="fancybox-inner")

featured_img = soup.find_all('img')[1]["src"]

final_img = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space' + featured_img
final_img

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Spaceimage/featured/mars3.jpg'

### Mars Facts

In [14]:
# Use Pandas to scrape data
mars_facts = pd.read_html('https://space-facts.com/mars/')

# Take first table for Mars facts
mars_facts = mars_facts[0]

# Rename columns and set index
mars_facts.columns=['Property', 'Value']
mars_facts

,Property,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


### Mars Hemispheres

In [9]:
# Open browser to USGS Astrogeology site
browser.visit('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')

In [13]:
# Parse HTML with Beautiful Soup
soup = bs(browser.html, 'html.parser')

# Create List to hold all the images & URLs collected
hems_image_url = []

#locate element that holds results for all hemis
results = soup.find_all('div', class_='item')

for result in results:
    # get tittle for hemis
    title_hemis = result.find('h3').text
    
    #to get the URL  - https://astrogeology.usgs.gov (29 char)+ href for full img (/search/map/Mars/Viking/cerberus_enhanced) 
    # = URL for the larger img page
    first_url = 'https://astrogeology.usgs.gov' + result.find('a', class_='itemLink')['href']
    
    #connect to first URL and parse
    browser.visit(first_url)
    soup = bs(browser.html, 'html.parser')
    
    # Open full image and get URL or src
    final_url = 'https://astrogeology.usgs.gov' + soup.find('img', class_='wide-image')['src']
    
    # Append list
    hems_image_url.append({'title': title_hemis, 'img_url':final_url})
    
    ##Go back
    browser.back()
    
hems_image_url

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]